In [2]:
import time

import torch
import torch.nn as nn
from torch import optim
from torchvision.models import resnet50
from torchsummary import summary
from torch.optim import lr_scheduler
from torch.cuda.amp import GradScaler, autocast

In [3]:
model = resnet50(pretrained=True).to('cuda:0')
# x = torch.rand(3, 640, 640).to('cuda:0')
# summary(model, x.squeeze(0).shape, batch_size=8)

D:\projects\cv-deep-learning\venv\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
D:\projects\cv-deep-learning\venv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:
targets = torch.ones(16, 1000).to(0)
def compute_loss(pred: torch.Tensor):
    func = nn.CrossEntropyLoss(reduction='mean')
    return func(pred, targets)


In [5]:
data = torch.rand(16, 3, 640, 640).to(0)
total_epoch = 5
def without_amp():
    optimizer = optim.SGD(model.parameters(), momentum=0.9, lr=0.1)
    scheduler = lr_scheduler.LambdaLR(optimizer, lambda x: (1 - x / total_epoch) * (1 - 0.1) + 0.1)

    for e in range(0, total_epoch):
        optimizer.zero_grad()
        pred = model(data)
        loss = compute_loss(pred)
        loss.backward()
        optimizer.step()
        scheduler.step()
t1 = time.time()
without_amp()
t2 = time.time()
print(f'不使用用amp消耗的时间：{t2 - t1}秒')

不使用用amp消耗的时间：3.076925277709961秒


In [9]:
def with_amp():
    optimizer = optim.SGD(model.parameters(), momentum=0.9, lr=0.1)
    scheduler = lr_scheduler.LambdaLR(optimizer, lambda x: (1 - x / total_epoch) * (1 - 0.1) + 0.1)
    scaler = GradScaler()

    for e in range(0, total_epoch):
        optimizer.zero_grad()
        # 对部分算子转为FP16(如Weight, Activations) , 部分算子转为FP32(拷贝一份)
        with autocast():
            pred = model(data)
            loss = compute_loss(pred)
        # 对loss缩放, 先转为FP16进行反向传播
        scaler.scale(loss).backward()
        # 更新权重前转为FP32， 如果loss为nan， 则跳过当前iteration权重更新
        scaler.step(optimizer)
        # 根据loss值动态修改scale factor
        scaler.update()
        scheduler.step()
t1 = time.time()
with_amp()
t2 = time.time()
print(f'使用amp消耗的时间：{t2 - t1}秒')

使用amp消耗的时间：1.554337978363037秒
